# REDUCING THE COMPUTATIONAL COMPLEXITY OF LEARNING WITH RANDOM CONVOLUTIONAL FEATURES


This Demo is an implementation of the E-ROCKET feature selection method for
removing redundancies of  [MINIROCKET featurs](https://github.com/angus924/minirocket) 

Please cite this work in the following manner:

`M. A. Omidi, B. Seyfe, and S. Valaee, “Reducing the computational
complexity of learning with random convolutional features,” in ICASSP
2023 - 2023 IEEE International Conference on Acoustics, Speech and
Signal Processing (ICASSP), 2023, pp. 1–5.`

[Click here to see EROCKET paper](https://ieeexplore.ieee.org/document/10095893) 

In [1]:
!pip install kneed
import sys
import numpy as np
#sys.path.append('/content/drive/MyDrive/erocket_analysis_codes/')
from erocket_module import feature_selector # experimental version
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifierCV

ucr_data_path = './'
ucr_data_name = 'Coffee'
#!git clone https://github.com/angus924/minirocket.git
sys.path.append('./minirocket-main/code/')
from minirocket import fit, transform


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:


training_data = np.loadtxt(ucr_data_path +
                                ucr_data_name +
                                '/'+ ucr_data_name + "_TRAIN.tsv")
Y_training, X_training = training_data[:, 0].astype(int), training_data[:, 1:].astype(np.float32)
#print('X_training.shape',X_training.shape)

test_data = np.loadtxt(ucr_data_path +
                                ucr_data_name +
                                '/'+ ucr_data_name + "_TEST.tsv")
Y_test, X_test = test_data[:, 0].astype(int), test_data[:, 1:].astype(np.float32)
#print('X_test.shape',X_test.shape)

parameters = fit(X_training)
X_training_transform = transform(X_training, parameters)
X_test_transform = transform(X_test, parameters)

sclr = StandardScaler()
sclr.fit(X_training_transform)
X_training_transform_scaled = sclr.transform(X_training_transform)
X_test_transform_scaled = sclr.transform(X_test_transform)
clf = RidgeClassifierCV(np.logspace(-3,3,10))
clf.fit(X_training_transform_scaled, Y_training)
acc_minirocket = clf.score(X_test_transform_scaled, Y_test)
w_ridgecv = clf.coef_

# Erocket accuracy
u_tilde, first_point_non_neg_weight, knees_minus, knees_plus = feature_selector(w_ridgecv)
X_training_transform_reduced = X_training_transform_scaled[:, u_tilde]
X_test_transform_reduced = X_test_transform_scaled[:, u_tilde]
clf_erocket = RidgeClassifierCV(np.logspace(-3,3,10))
clf_erocket.fit(X_training_transform_reduced, Y_training)
acc_erocket = clf_erocket.score(X_test_transform_reduced, Y_test)
print('=' * 50)
print('Minirocket')
print('Accuracy:', acc_minirocket, 'Number of features:', X_training_transform.shape[1])
print('=' * 50)
print('EROCKET')
print('Accuracy:', acc_erocket, 'Number of features:', X_training_transform_reduced.shape[1])

Minirocket
Accuracy: 1.0 Number of features: 9996
EROCKET
Accuracy: 1.0 Number of features: 389
